In [3]:
import warnings
import pprint
import skrebate
import imblearn
from imblearn import under_sampling, over_sampling, combine
from imblearn.pipeline import Pipeline as imbPipeline
from sklearn import (preprocessing, svm, linear_model, ensemble, naive_bayes,
                    tree, neighbors, decomposition, kernel_approximation, cluster)
from sklearn.pipeline import Pipeline
from sklearn.base import clone

from sklearn.compose import TransformedTargetRegressor
from sklearn.model_selection import (KFold, GroupKFold, StratifiedKFold,
                                    LeaveOneGroupOut, cross_validate,
                                    cross_val_predict, learning_curve,
                                    GridSearchCV)
from sklearn.feature_selection import SelectKBest, f_regression, SelectFromModel, VarianceThreshold, f_classif
from sklearn.metrics import (r2_score, auc, roc_auc_score, balanced_accuracy_score, 
                             average_precision_score, confusion_matrix, roc_curve,
                             precision_recall_curve)
from sklearn.metrics.scorer import roc_auc_scorer
from sklearn.preprocessing import QuantileTransformer, quantile_transform, StandardScaler, MinMaxScaler
from sklearn.utils.class_weight import compute_class_weight, compute_sample_weight
from sklearn.utils.validation import check_memory
from xgboost import XGBRegressor, XGBClassifier
from sklearn.ensemble import RandomForestClassifier

warnings.simplefilter('ignore')

/Users/guq/miniconda3/envs/python3/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)
/Users/guq/miniconda3/envs/python3/lib/python3.7/site-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [4]:
import os
import sys

In [5]:
import numpy as np
import pandas as pd
import re

import plotly.io as pio
import plotly.plotly as py
import plotly.graph_objs as go
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

# result

In [6]:
work_dir = '/Users/guq/Documents/ohsu/drug_respond/results/smmart_nestedCV_2019/'
sub1_1 = 'Hyperparameter_xgbc_tree_d/'
sub1_2 = 'Hyperparameter_xgbc_linear_d/'
sub1_3 = 'Hyperparameter_xgbc_dart_d/'
sub2_1 = 'Hyperparameter_xgbr_tree_d/'
sub2_2 = 'Hyperparameter_xgbr_linear_d/'
sub2_3 = 'Hyperparameter_xgbr_dart_d/'
sub3 = 'Hyperparameter_rfc_d/'
"""sub4 = 'Hyperparameter_iraps_d'"""
sub5 = 'Hyperparameter_ridge_d'
sub6 = 'Hyperparameter_knr_d'
sub7 = 'Hyperparameter_svr_d'

In [7]:
smmart_drugs = ('Afatinib', 'Bortezomib', 'Cabozantinib', 'Dasatinib', 'Erlotinib',
                'Idelalisib', 'Imatinib', 'Olaparib', 'Palbociclib', 'Ponatinib',
                'Rapamycin', 'Sorafenib', 'Sunitinib', 'Trametinib', 'Vorinostat')

def concate_best_result(folder, file_name, scorer, classifier, results, discretize_z_score=-1, exact_z_score=False):
    
    if not file_name.endswith('tabular'):
        return results
    path = os.path.join(folder, file_name)
    res = pd.read_csv(path, sep='\t')
    
    res_filtered = res
    if exact_z_score:
        #print(path)
        if classifier == 'IRAPSClassifier':
            res_filtered = res_filtered[res_filtered['param_estimator__discretize'] == discretize_z_score]
            if discretize_z_score != -1:
                classifier = 'IRAPSClassifier_05'
        else:
            res_filtered = res_filtered[res_filtered['param_estimator__z_score'] == discretize_z_score]
            if discretize_z_score != -1:
                classifier = classifier + '_05'
        #print(classifier)
    res_sort = res_filtered.sort_values(['mean_test_'+scorer, 'std_test_'+scorer], ascending=[False, True])
    res_best = res_sort[['mean_test_'+scorer, 'std_test_'+scorer,'params']].head(1).reset_index(drop=True)
    if not res_best['mean_test_'+scorer].isnull().any():
        if file_name.startswith((smmart_drugs)):
            res_best.insert(loc=0, column='dataset', value=file_name[:-8])
        else:
            res_best.insert(loc=0, column='dataset', value=file_name[:-11])
        res_best.insert(loc=0, column='classifier', value=classifier)
        if results is None:
            results = res_best
        else:
            results = results.append(res_best, ignore_index=True)
    return results

In [8]:
# best AP scores
files1_1 = os.listdir(work_dir+sub1_1)
files1_2 = os.listdir(work_dir+sub1_2)
files1_3 = os.listdir(work_dir+sub1_3)
files2_1 = os.listdir(work_dir+sub2_1)
files2_2 = os.listdir(work_dir+sub2_2)
files2_3 = os.listdir(work_dir+sub2_3)
files3 = os.listdir(work_dir+sub3)
"""files4 = os.listdir(work_dir+sub4)"""
files5 = os.listdir(work_dir+sub5)
files6 = os.listdir(work_dir+sub6)
files7 = os.listdir(work_dir+sub7)
results = None
scorer = 'binarize_average_precision_scorer'
for fl in files1_1:
    results = concate_best_result(work_dir+sub1_1, fl, scorer, 'XGBClassifier_tree_d', results)
    
for fl in files1_2:
    results = concate_best_result(work_dir+sub1_2, fl, scorer, 'XGBClassifier_linear_d', results)

for fl in files1_3:
    results = concate_best_result(work_dir+sub1_3, fl, scorer, 'XGBClassifier_dart_d', results)

for fl in files2_1:
    results = concate_best_result(work_dir+sub2_1, fl, scorer, 'XGBRegressor_tree_d', results)

for fl in files2_2:
    results = concate_best_result(work_dir+sub2_2, fl, scorer, 'XGBRegressor_linear_d', results)
    
for fl in files2_3:
    results = concate_best_result(work_dir+sub2_3, fl, scorer, 'XGBRegressor_dart_d', results)
    
for fl in files3:
    results = concate_best_result(work_dir+sub3, fl, scorer, 'RandomForestClassifier_d', results)

"""for fl in files4:
    results = concate_best_result(work_dir+sub4, fl, scorer, 'IRAPS', results)"""
    
for fl in files5:
    results = concate_best_result(work_dir+sub5, fl, scorer, 'Ridge_d', results)
    
for fl in files6:
    results = concate_best_result(work_dir+sub6, fl, scorer, 'KNeighborsRegressor_d', results)

for fl in files7:
    results = concate_best_result(work_dir+sub7, fl, scorer, 'SVR_d', results)


results = results.sort_values(['classifier', 'dataset'])
results

,classifier,dataset,mean_test_binarize_average_precision_scorer,std_test_binarize_average_precision_scorer,params
146,KNeighborsRegressor_d,Afatinib_1032,0.371848,0.122198,{'regressor__selectkbest': None}
138,KNeighborsRegressor_d,Afatinib_1377,0.430708,0.075090,{'regressor__selectkbest': None}
139,KNeighborsRegressor_d,Bortezomib_104,0.309675,0.102929,{'regressor__selectkbest': None}
142,KNeighborsRegressor_d,Cabozantinib_249,0.284119,0.064745,{'regressor__selectkbest': None}
147,KNeighborsRegressor_d,Dasatinib_51,0.409879,0.129274,{'regressor__selectkbest': None}
137,KNeighborsRegressor_d,Erlotinib_1,0.439573,0.282145,{'regressor__selectkbest': None}
143,KNeighborsRegressor_d,Idelalisib_238,0.308184,0.085615,{'regressor__selectkbest': None}
144,KNeighborsRegressor_d,Imatinib_34,0.424327,0.175659,{'regressor__selectkbest': None}
152,KNeighborsRegressor_d,Olaparib_1017,0.273013,0.089793,{'regressor__selectkbest': None}
145,KNeighborsRegressor_d,Olaparib_1495,0.267482,0.086816,{'regressor__selectkbest': None}


In [9]:
# 

In [10]:
# best roc-auc scores
files1_1 = os.listdir(work_dir+sub1_1)
files1_2 = os.listdir(work_dir+sub1_2)
files1_3 = os.listdir(work_dir+sub1_3)
files2_1 = os.listdir(work_dir+sub2_1)
files2_2 = os.listdir(work_dir+sub2_2)
files2_3 = os.listdir(work_dir+sub2_3)
files3 = os.listdir(work_dir+sub3)
"""files4 = os.listdir(work_dir+sub4)"""
files5 = os.listdir(work_dir+sub5)
files6 = os.listdir(work_dir+sub6)
files7 = os.listdir(work_dir+sub7)
results_auc = None
scorer = 'binarize_auc_scorer'
for fl in files1_1:
    results_auc = concate_best_result(work_dir+sub1_1, fl, scorer, 'XGBClassifier_tree_d', results_auc)
    
for fl in files1_2:
    results_auc = concate_best_result(work_dir+sub1_2, fl, scorer, 'XGBClassifier_linear_d', results_auc)

for fl in files1_3:
    results_auc = concate_best_result(work_dir+sub1_3, fl, scorer, 'XGBClassifier_dart_d', results_auc)

for fl in files2_1:
    results_auc = concate_best_result(work_dir+sub2_1, fl, scorer, 'XGBRegressor_tree_d', results_auc)

for fl in files2_2:
    results_auc = concate_best_result(work_dir+sub2_2, fl, scorer, 'XGBRegressor_linear_d', results_auc)
    
for fl in files2_3:
    results_auc = concate_best_result(work_dir+sub2_3, fl, scorer, 'XGBRegressor_dart_d', results_auc)
    
for fl in files3:
    results_auc = concate_best_result(work_dir+sub3, fl, scorer, 'RandomForestClassifier_d', results_auc)

"""for fl in files4:
    results = concate_best_result(work_dir+sub4, fl, scorer, 'IRAPS', results)"""
    
for fl in files5:
    results_auc = concate_best_result(work_dir+sub5, fl, scorer, 'Ridge_d', results_auc)
    
for fl in files6:
    results_auc = concate_best_result(work_dir+sub6, fl, scorer, 'KNeighborsRegressor_d', results_auc)

for fl in files7:
    results_auc = concate_best_result(work_dir+sub7, fl, scorer, 'SVR_d', results_auc)

results_auc = results_auc.sort_values(['classifier', 'dataset'])
results_auc

,classifier,dataset,mean_test_binarize_auc_scorer,std_test_binarize_auc_scorer,params
146,KNeighborsRegressor_d,Afatinib_1032,0.838616,0.042457,{'regressor__selectkbest': None}
138,KNeighborsRegressor_d,Afatinib_1377,0.833016,0.030864,{'regressor__selectkbest': None}
139,KNeighborsRegressor_d,Bortezomib_104,0.631525,0.103801,{'regressor__selectkbest': None}
142,KNeighborsRegressor_d,Cabozantinib_249,0.668580,0.069852,{'regressor__selectkbest': None}
147,KNeighborsRegressor_d,Dasatinib_51,0.730055,0.048149,{'regressor__selectkbest': None}
137,KNeighborsRegressor_d,Erlotinib_1,0.744433,0.186209,{'regressor__selectkbest': None}
143,KNeighborsRegressor_d,Idelalisib_238,0.773420,0.063617,{'regressor__selectkbest': None}
144,KNeighborsRegressor_d,Imatinib_34,0.725447,0.188214,{'regressor__selectkbest': None}
152,KNeighborsRegressor_d,Olaparib_1017,0.634888,0.050995,{'regressor__selectkbest': None}
145,KNeighborsRegressor_d,Olaparib_1495,0.668774,0.100521,{'regressor__selectkbest': None}


In [13]:
data1_1 = go.Bar(
    x = results[results['classifier'] == 'XGBClassifier_tree_d']['dataset'],
    y = results[results['classifier'] == 'XGBClassifier_tree_d']['mean_test_binarize_average_precision_scorer'],
    error_y = dict(
        type='data',
        array=results[results['classifier'] == 'XGBClassifier_tree_d']['std_test_binarize_average_precision_scorer']),  
    name = 'xgbc_tree_AP'
)

data1_2 = go.Bar(
    x = results[results['classifier'] == 'XGBClassifier_linear_d']['dataset'],
    y = results[results['classifier'] == 'XGBClassifier_linear_d']['mean_test_binarize_average_precision_scorer'],
    error_y = dict(
        type='data',
        array=results[results['classifier'] == 'XGBClassifier_linear_d']['std_test_binarize_average_precision_scorer']),
    name = 'xgbc_linar_AP'
)

data1_3 = go.Bar(
    x = results[results['classifier'] == 'XGBClassifier_dart_d']['dataset'],
    y = results[results['classifier'] == 'XGBClassifier_dart_d']['mean_test_binarize_average_precision_scorer'],
    error_y = dict(
        type='data',
        array=results[results['classifier'] == 'XGBClassifier_dart_d']['std_test_binarize_average_precision_scorer']),
    name = 'xgbc_dart_AP'
)

data2_1 = go.Bar(
    x = results[results['classifier'] == 'XGBRegressor_tree_d']['dataset'],
    y = results[results['classifier'] == 'XGBRegressor_tree_d']['mean_test_binarize_average_precision_scorer'],
    error_y = dict(
        type='data',
        array=results[results['classifier'] == 'XGBRegressor_tree_d']['std_test_binarize_average_precision_scorer']),
    name = 'xgbr_tree_AP'
)

data2_2 = go.Bar(
    x = results[results['classifier'] == 'XGBRegressor_linear_d']['dataset'],
    y = results[results['classifier'] == 'XGBRegressor_linear_d']['mean_test_binarize_average_precision_scorer'],
    error_y = dict(
        type='data',
        array=results[results['classifier'] == 'XGBRegressor_linear_d']['std_test_binarize_average_precision_scorer']),
    name = 'xgbr_linar_AP'
)

data2_3 = go.Bar(
    x = results[results['classifier'] == 'XGBRegressor_dart_d']['dataset'],
    y = results[results['classifier'] == 'XGBRegressor_dart_d']['mean_test_binarize_average_precision_scorer'],
    error_y = dict(
        type='data',
        array=results[results['classifier'] == 'XGBRegressor_dart_d']['std_test_binarize_average_precision_scorer']),
    name = 'xgbr_dart_AP'
)

data3 = go.Bar(
    x = results[results['classifier'] == 'RandomForestClassifier_d']['dataset'],
    y = results[results['classifier'] == 'RandomForestClassifier_d']['mean_test_binarize_average_precision_scorer'],
    error_y = dict(
        type='data',
        array=results[results['classifier'] == 'RandomForestClassifier_d']['std_test_binarize_average_precision_scorer']),
    name = 'rfc_AP'
)

data5 = go.Bar(
    x = results[results['classifier'] == 'Ridge_d']['dataset'],
    y = results[results['classifier'] == 'Ridge_d']['mean_test_binarize_average_precision_scorer'],
    error_y = dict(
        type='data',
        array=results[results['classifier'] == 'Ridge_d']['std_test_binarize_average_precision_scorer']),
    name = 'ridge_AP'
)

data6 = go.Bar(
    x = results[results['classifier'] == 'KNeighborsRegressor_d']['dataset'],
    y = results[results['classifier'] == 'KNeighborsRegressor_d']['mean_test_binarize_average_precision_scorer'],
    error_y = dict(
        type='data',
        array=results[results['classifier'] == 'KNeighborsRegressor_d']['std_test_binarize_average_precision_scorer']),
    name = 'knr_AP'
)

data7 = go.Bar(
    x = results[results['classifier'] == 'SVR_d']['dataset'],
    y = results[results['classifier'] == 'SVR_d']['mean_test_binarize_average_precision_scorer'],
    error_y = dict(
        type='data',
        array=results[results['classifier'] == 'SVR_d']['std_test_binarize_average_precision_scorer']),
    name = 'svr_AP'
)

data11_1 = go.Bar(
    x = results_auc[results_auc['classifier'] == 'XGBClassifier_tree_d']['dataset'],
    y = results_auc[results_auc['classifier'] == 'XGBClassifier_tree_d']['mean_test_binarize_auc_scorer'],
    error_y = dict(
        type='data',
        array=results_auc[results_auc['classifier'] == 'XGBClassifier_tree_d']['std_test_binarize_auc_scorer']),
    name = 'xgbc_tree_ROC'
)

data11_2 = go.Bar(
    x = results_auc[results_auc['classifier'] == 'XGBClassifier_linear_d']['dataset'],
    y = results_auc[results_auc['classifier'] == 'XGBClassifier_linear_d']['mean_test_binarize_auc_scorer'],
    error_y = dict(
        type='data',
        array=results_auc[results_auc['classifier'] == 'XGBClassifier_linear_d']['std_test_binarize_auc_scorer']),
    name = 'xgbc_linar_ROC'
)

data11_3 = go.Bar(
    x = results_auc[results_auc['classifier'] == 'XGBClassifier_dart_d']['dataset'],
    y = results_auc[results_auc['classifier'] == 'XGBClassifier_dart_d']['mean_test_binarize_auc_scorer'],
    error_y = dict(
        type='data',
        array=results_auc[results_auc['classifier'] == 'XGBClassifier_dart_d']['std_test_binarize_auc_scorer']),
    name = 'xgbc_dart_ROC'
)

data12_1 = go.Bar(
    x = results_auc[results_auc['classifier'] == 'XGBRegressor_tree_d']['dataset'],
    y = results_auc[results_auc['classifier'] == 'XGBRegressor_tree_d']['mean_test_binarize_auc_scorer'],
    error_y = dict(
        type='data',
        array=results_auc[results_auc['classifier'] == 'XGBRegressor_tree_d']['std_test_binarize_auc_scorer']),
    name = 'xgbr_tree_ROC'
)

data12_2 = go.Bar(
    x = results_auc[results_auc['classifier'] == 'XGBRegressor_linear_d']['dataset'],
    y = results_auc[results_auc['classifier'] == 'XGBRegressor_linear_d']['mean_test_binarize_auc_scorer'],
    error_y = dict(
        type='data',
        array=results_auc[results_auc['classifier'] == 'XGBRegressor_linear_d']['std_test_binarize_auc_scorer']),
    name = 'xgbr_linar_ROC'
)

data12_3 = go.Bar(
    x = results_auc[results_auc['classifier'] == 'XGBRegressor_dart_d']['dataset'],
    y = results_auc[results_auc['classifier'] == 'XGBRegressor_dart_d']['mean_test_binarize_auc_scorer'],
    error_y = dict(
        type='data',
        array=results_auc[results_auc['classifier'] == 'XGBRegressor_dart_d']['std_test_binarize_auc_scorer']),
    name = 'xgbr_dart_ROC'
)

data13 = go.Bar(
    x = results_auc[results_auc['classifier'] == 'RandomForestClassifier_d']['dataset'],
    y = results_auc[results_auc['classifier'] == 'RandomForestClassifier_d']['mean_test_binarize_auc_scorer'],
    error_y = dict(
        type='data',
        array=results_auc[results_auc['classifier'] == 'RandomForestClassifier_d']['std_test_binarize_auc_scorer']),
    name = 'rfc_ROC'
)

data15 = go.Bar(
    x = results_auc[results_auc['classifier'] == 'Ridge_d']['dataset'],
    y = results_auc[results_auc['classifier'] == 'Ridge_d']['mean_test_binarize_auc_scorer'],
    error_y = dict(
        type='data',
        array=results_auc[results_auc['classifier'] == 'Ridge_d']['std_test_binarize_auc_scorer']),
    name = 'ridge_ROC'
)

data16 = go.Bar(
    x = results_auc[results_auc['classifier'] == 'KNeighborsRegressor_d']['dataset'],
    y = results_auc[results_auc['classifier'] == 'KNeighborsRegressor_d']['mean_test_binarize_auc_scorer'],
    error_y = dict(
        type='data',
        array=results_auc[results_auc['classifier'] == 'KNeighborsRegressor_d']['std_test_binarize_auc_scorer']),
    name = 'knr_ROC'
)

data17 = go.Bar(
    x = results_auc[results_auc['classifier'] == 'SVR_d']['dataset'],
    y = results_auc[results_auc['classifier'] == 'SVR_d']['mean_test_binarize_auc_scorer'],
    error_y = dict(
        type='data',
        array=results_auc[results_auc['classifier'] == 'SVR_d']['std_test_binarize_auc_scorer']),
    name = 'svr_ROC'
)



layout = go.Layout(
    xaxis=dict(
        title='Dataset'
    ),
    yaxis=dict(
        title='Performance score'
    ),
    barmode = 'group'
)
fig = go.Figure(data=[data1_1, data1_2, data1_3, data2_1, data2_2, data2_3, data3, data5, data6, data7], layout=layout)
iplot(fig)
#iplot(fig, filename='/Users/guq/Downloads/newplot_ap_1.png', image='png')


fig = go.Figure(data=[data11_1, data11_2, data11_3, data12_1, data12_2, data12_3, data13, data15, data16, data17], layout=layout)
#iplot(fig, filename='/Users/guq/Downloads/newplot_roc_1.png', image='png')
iplot(fig)
# To show plot, paste the link to this GitHub notebook into http://nbviewer.jupyter.org/

In [14]:
trace1 = {
    "type": 'violin',
    "x": results['classifier'],
    "y": results['mean_test_binarize_average_precision_scorer'],
    "legendgroup": 'AP',
    "scalegroup": 'AP',
    "name": 'AP',
    "box": {
        "visible": True
    },
    "meanline": {
        "visible": True
    },
    "line": {
        "color": 'blue'
    }
}

trace2 = {
    "type": 'violin',
    "x": results_auc['classifier'],
    "y": results_auc['mean_test_binarize_auc_scorer'],
    "legendgroup": 'ROC-AUC',
    "scalegroup": 'ROC-AUC',
    "name": 'ROC-AUC',
    "box": {
        "visible": True
    },
    "meanline": {
        "visible": True
    },
    "line": {
        "color": 'pink'
    }
}

layout = {
    "yaxis": {
        "zeroline": False,
    },
    "violinmode": 'group'
}
fig = go.Figure(data=[trace1, trace2], layout=layout)
iplot(fig)
# To show plot, paste the link to this GitHub notebook into http://nbviewer.jupyter.org/